# DeepCausalMMM Quick Start Guide

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adityapt/deepcausalmmm/blob/main/examples/quickstart.ipynb)
[![PyPI](https://badge.fury.io/py/deepcausalmmm.svg)](https://pypi.org/project/deepcausalmmm/)

Welcome! This notebook demonstrates **DeepCausalMMM** for Marketing Mix Modeling. If Colab throws Numpy Error run this in your local.

## What You'll Learn

1. Install DeepCausalMMM
2. Generate synthetic MMM data
3. Train a model
4. Analyze results

## Step 1: Installation

In [ ]:
!pip install deepcausalmmm -q

## Step 2: Import Libraries

In [ ]:
import numpy as np
import deepcausalmmm

from deepcausalmmm.core import get_default_config
from deepcausalmmm.core.trainer import ModelTrainer
from deepcausalmmm.core.data import UnifiedDataPipeline
from deepcausalmmm.utils.data_generator import ConfigurableDataGenerator

print(f" DeepCausalMMM v{deepcausalmmm.__version__} loaded!")

## Step 3: Generate Synthetic Data

Generate realistic MMM data with:
- 10 regions (DMAs)
- 52 weeks
- 5 media channels
- 3 control variables

In [ ]:
# Get config
config = get_default_config()
config['n_epochs'] = 200  # Reduce for quick demo
config['random_seed'] = 42

# Generate data
generator = ConfigurableDataGenerator(config)
X_media, X_control, y = generator.generate_mmm_dataset(
    n_regions=10,
    n_weeks=52,
    n_media_channels=5,
    n_control_channels=3
)

print(f" Data generated:")
print(f"   Media: {X_media.shape}")
print(f"   Control: {X_control.shape}")
print(f"   Target: {y.shape}")

## Step 4: Split and Process Data

In [ ]:
# Initialize pipeline
pipeline = UnifiedDataPipeline(config)

# Split into train/holdout
train_data, holdout_data = pipeline.temporal_split(X_media, X_control, y)

# Process training data
train_tensors = pipeline.fit_and_transform_training(train_data)

print(f" Data processed:")
print(f"   Training: {train_tensors['X_media'].shape}")

## Step 5: Create and Train Model

In [ ]:
# Create trainer
trainer = ModelTrainer(config)

# Get dimensions
n_media = train_tensors['X_media'].shape[2]
n_control = train_tensors['X_control'].shape[2]
n_regions = train_tensors['X_media'].shape[0]

# Create model
model = trainer.create_model(n_media, n_control, n_regions)
trainer.create_optimizer_and_scheduler()

# Prepare full data for baseline
full_data = {'X_media': X_media, 'X_control': X_control, 'y': y}
full_tensors = pipeline.fit_and_transform_training(full_data)

# Train
print(" Training model...")
results = trainer.train(
    train_tensors['X_media'], train_tensors['X_control'],
    train_tensors['R'], train_tensors['y'],
    y_full_for_baseline=full_tensors['y'],
    verbose=True
)

print("\n Training complete!")

## Step 6: View Results

In [ ]:
print(" Performance Metrics:\n")
print(f"Final Training R²: {results.get('final_train_r2', 0):.4f}")
print(f"Final Training RMSE: {results.get('final_train_rmse', 0):,.0f}")
print(f"\nBest RMSE (log-space): {results.get('best_rmse', float('inf')):.4f}")

# Show holdout metrics if available
if 'final_holdout_r2' in results and results['final_holdout_r2'] is not None:
    print(f"\n Holdout Metrics:")
    print(f"   R²: {results['final_holdout_r2']:.4f}")
    print(f"   RMSE: {results['final_holdout_rmse']:,.0f}")

## Step 7: Get Predictions

In [ ]:
# Get predictions
model.eval()
import torch

with torch.no_grad():
    outputs = model(
        train_tensors['X_media'],
        train_tensors['X_control'],
        train_tensors['R']
    )
    predictions = outputs[0]
    media_contrib = outputs[1]

print(f" Predictions: {predictions.shape}")
print(f" Media contributions: {media_contrib.shape}")

## Step 8: Analyze Channels

In [ ]:
import pandas as pd

# Channel contributions
channel_totals = media_contrib.sum(dim=(0, 1)).cpu().numpy()
channels = [f"Channel_{i+1}" for i in range(len(channel_totals))]

contrib_df = pd.DataFrame({
    'Channel': channels,
    'Contribution': channel_totals,
    'Percentage': channel_totals / channel_totals.sum() * 100
}).sort_values('Contribution', ascending=False)

print(" Channel Impact:\n")
print(contrib_df.to_string(index=False))

##  Congratulations!

You've successfully:
-  Installed DeepCausalMMM
-  Generated synthetic data
-  Trained a model
-  Analyzed channel contributions

## Next Steps

###  Learn More:
- [Documentation](https://deepcausalmmm.readthedocs.io/)
- [API Reference](https://deepcausalmmm.readthedocs.io/en/latest/api/)
- [GitHub](https://github.com/adityapt/deepcausalmmm)

###  Advanced Features:
- **Response Curves**: Analyze saturation effects
- **DAG Discovery**: Explore causal relationships
- **Multi-Region Analysis**: Compare regional performance
- **Comprehensive Dashboard**: Interactive visualizations
- **Hyperparameter Tuning**: Optimize model performance

### Citation:
```bibtex
@article{tirumala2025deepcausalmmm,
  title={DeepCausalMMM: A Deep Learning Framework for Marketing Mix Modeling with Causal Inference},
  author={Puttaparthi Tirumala, Aditya},
  journal={arXiv preprint arXiv:2510.13087},
  year={2025}
}
```

**Happy Modeling/Users/adityapu/Documents/GitHub/deepcausalmmm && grep -n zenodo|version|1.0.17 examples/quickstart.ipynb | head -20*